In [ ]:
def read_file(file):
    import csv
    with open(file, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        header = next(reader)
        #     names = [line[0] for line in reader]
        #     emails = [line[1] for line in reader] generator can only be used once
        names = []
        emails = []
        for line in reader:
            names.append(line[0])
            emails.append(line[1])
    return names, emails

In [ ]:
read_file('contacts.csv')

In [ ]:
def read_template(message):
    from string import Template
    with open(message, 'r', encoding='utf-8') as f:
        content = f.read()
    return Template(content)

In [ ]:
read_template('message.txt')

In [1]:
import smtplib
from mail_data import EMAIL, PASS

In [5]:
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()

(220, b'2.0.0 Ready to start TLS')

In [6]:
server.login(EMAIL, PASS)

(235, b'2.7.0 Accepted')